In [1]:
import math
import numpy as np
import networkx as nx
import pandas as pd
from collections import Counter
import operator

import config
from config import load_users_dataframe
from config import load_networkx_all
from config import load_networkx_friends
from config import load_networkx_potential

# Plotting
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Networkx
from networkx.drawing.nx_agraph import graphviz_layout
import dzcnapy_plotlib as dzcnapy

project_name = config.settings['data']['eventname']
timeframe = config.settings['timeframe']
save_to_file = config.settings['save_to_file'] == 'True'
window = 30
unique_users = load_users_dataframe()
network_all = load_networkx_all()
network_friends = load_networkx_friends()
network_potential = load_networkx_potential()

{'calculate': {'analysis': True,
               'friends': False,
               'network': True,
               'uniquetweets': True,
               'uniqueusers': True},
 'data': {'dates': ['2018-03-13', '2018-03-14', '2018-03-15'],
          'eventname': "hawking's death",
          'phrases': ['stephen%20hawking%20death',
                      'stephen%20hawking%20die',
                      'stephen%20hawking%20passed%20away'],
          'starttime': 'Mar 13 12:00:00 -0500 2018'},
 'path': {'cwd': '/home/ec2-user/uclresearchanalysis/data/hawking',
          'networkx': {'all': '/home/ec2-user/uclresearchanalysis/data/hawking/pickle/networkx_all.dat',
                       'friends': '/home/ec2-user/uclresearchanalysis/data/hawking/pickle/networkx_friends.dat',
                       'potential': '/home/ec2-user/uclresearchanalysis/data/hawking/pickle/networkx_potential.dat'},
          'newcrawl': '/home/ec2-user/uclresearchanalysis/other/newcrawl.dat',
          'pickle': {'frie

# Graphviz Notes
## Program
neato - [Spring][Undirected]

dot - [Directed][DAG][Hierarchies]

twopi - [Radial layout]

circo - [Circular layout]

fdp - [Spring][Undirected]

nop - [Stream of graphs]

acyclic - [Returns ascyclic graphy by reverting edges]

gvpr - [Graph stream editor]

gvcolor - [A filter that sets node colors from initial seed values. Colors flow along edges from tail to head, and are averaged as HSB vector at nodes]

ccomps - [Decompose to connected components]

sccmap - [Decompose digraphs into strongly connected components]

tred - [Transitive reduction of directed graphs]

sfdp - [Spring][Undirected]


In [2]:
def plot_network_using_pos(title, graph, pos, nodeColor = None, edgeColor = None, patchList = []):
    attributes = dzcnapy.small_attrs.copy()
    if nodeColor is not None:
        attributes["node_color"] = nodeColor
    if edgeColor is not None:
        attributes["edge_color"] = edgeColor
    nx.draw_networkx(graph, pos=pos, with_labels=False, **attributes)
    plt.legend(handles=patchList)
    plt.axis('off')
    dzcnapy.set_extent(pos, plt)
    dzcnapy.plot(title, [30, 30], save=True)
    
def plot_network(title, graph, prog, args, nodeColor = None, edgeColor = None, patchList = []):
    pos = graphviz_layout(graph, prog=prog, args=args)
    plot_network_using_pos(title, graph, pos, nodeColor, edgeColor, patchList)

In [3]:
def get_colors(depth):
    return sns.cubehelix_palette(depth, start=3, rot=1, reverse=True)
    sns.palplot(colors)

# Plot spreading network

In [4]:
%%time
def plot_spreading_network():
    depth = max(nx.get_node_attributes(network_all,'generation').values()) + 1
    palette = get_colors(depth)
    colors = [palette[network_all.node[n]["generation"]] for n in network_all]
    patchList = []
    for index in range(depth):
        data_key = mpatches.Patch(color=palette[index], label='Generation {}'.format(index))
        patchList.append(data_key)
    plot_network('Spreading network [spring] for event {}'.format(project_name), 
                 network_all, 'sfdp', '-Goverlap=scale', nodeColor = colors, patchList = patchList)
    plot_network('Spreading network [hierachy] for event {}'.format(project_name), 
                 network_all, 'dot', '-Goverlap=scale', nodeColor = colors, patchList = patchList)
plot_spreading_network()

/home/ec2-user/miniconda3/lib/python3.6/site-packages/pygraphviz/agraph.py:1341: RuntimeWarning: Error: remove_overlap: Graphviz not built with triangulation library

  warnings.warn(b"".join(errors).decode(self.encoding), RuntimeWarning)
/home/ec2-user/miniconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:1297: UserWarning: findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


KeyboardInterrupt: 

In [4]:
network_sfdp_pos = graphviz_layout(network_all, prog='sfdp', args='-Goverlap=scale')
# network_dot_pos = graphviz_layout(network_all, prog='dot', args='-Goverlap=scale')

/home/ec2-user/miniconda3/lib/python3.6/site-packages/pygraphviz/agraph.py:1341: RuntimeWarning: Error: remove_overlap: Graphviz not built with triangulation library

  warnings.warn(b"".join(errors).decode(self.encoding), RuntimeWarning)


# Plot friends network

In [5]:
%%time
friends_pos = graphviz_layout(network_friends, prog='sfdp', args='-Goverlap=scale')

OSError: Error: remove_overlap: Graphviz not built with triangulation library
out of memory


In [6]:
plot_network_using_pos(
    'Friendship network [spring] for event {}'.format(project_name), 
    network_friends, friends_pos)

NameError: name 'friends_pos' is not defined

# Seeds

In [7]:
def plot_seed_spreading(number):
    topSeeds = list(unique_users.seed_index.value_counts()[0:number].index)
    for idx, seed_index in enumerate(topSeeds):
        shortest_paths = nx.shortest_path(network_friends, source=seed_index)
        getDistance = lambda x: len(shortest_paths[x]) - 1 if x in shortest_paths else -1
        distances = [getDistance(x) for x in network_all]
        distances_counter = Counter(distances)
        palette = get_colors(len(distances_counter))
        getColor = lambda x: palette[(len(shortest_paths[x]) - 1)] if x in shortest_paths else palette[-1]
        colors = [getColor(x) for x in network_all]
        patchList = []
        for index in range(len(distances_counter) - 1):
            data_key = mpatches.Patch(color=palette[index], label='Generation {}, count {}'.format(index, distances_counter[index]))
            patchList.append(data_key)
        patchList.append(mpatches.Patch(color=palette[-1], label='Other, count {}'.format(distances_counter[-1])))
        plot_network_using_pos(
            '{}, Seed {} Spreading network [spring] for event {} for '.format(idx, seed_index, project_name), 
            network_all, network_sfdp_pos, nodeColor = colors, patchList = patchList)

In [8]:
%%time
plot_seed_spreading(10)

/home/ec2-user/miniconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:1297: UserWarning: findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


CPU times: user 1h 29min 55s, sys: 19.6 s, total: 1h 30min 14s
Wall time: 1h 30min 6s
